In [1]:
import numpy as np
import yaml
import random
import itertools
import os

In [2]:
def create_player_dict(ranking):
    player_dict = {}
    player_ids = []

    for idx, pid in enumerate(ranking):
        player_dict[pid] = idx
        player_ids.append(pid)
        
    return player_dict, player_ids

In [3]:
def create_ranking_distance_matrix(player_count):
    ranking_distance_matrix = np.zeros((player_count, player_count), dtype=int)

    for i in range(player_count):
        for y in range(player_count):
            ranking_distance_matrix[i, y] = abs(i-y)
            ranking_distance_matrix[y, i] = abs(i-y)
            
    return ranking_distance_matrix

In [4]:
def create_already_played_matrix(player_count, completed_matches):
    already_played_matrix = np.zeros((player_count, player_count), dtype=int)

    if completed_matches is not None:
        for m in completed_matches:
            p1, p2 = m.strip().split(';', maxsplit=2)
            p1 = int(p1.strip())
            p2 = int(p2.strip())

            already_played_matrix[player_dict[p1], player_dict[p2]] += 1
            already_played_matrix[player_dict[p2], player_dict[p1]] += 1
            
    return already_played_matrix

In [5]:
def create_groups(player_count, group_size):
    ranking = list(range(player_count))
    c_group = 0
    c_group_size = 0
    groups = [[]]

    for player in ranking:
        c_group_size +=1
        groups[c_group].append(player)

        if c_group_size == group_size:
            c_group_size = 0
            c_group += 1

            if player < player_count-1:
                groups.append([])
    
    return groups

In [6]:
def initialize_matchmaking_matrix(player_count, groups):
    matchmaking_matrix = np.zeros((player_count, player_count), dtype=int)
    fixed_group_matches = 0
    
    for g in groups:
        for p1 in g:
            for p2 in g:                
                if p1 != p2:
                    matchmaking_matrix[p1, p2] = 1
                
                if p1 < p2:
                    fixed_group_matches += 1
    
    return matchmaking_matrix, fixed_group_matches

In [7]:
def get_possible_opponents(matchmaking_matrix):
    player_count = matchmaking_matrix.shape[0]
    possible_opponents = []
    
    for i in range(player_count):
        possible_opponents.append([])
        
    for p1 in range(player_count):
        for p2 in range(player_count):
            if p1 != p2:
                if matchmaking_matrix[p1, p2] == 0:
                    possible_opponents[p1].append(p2)
    
    return possible_opponents

In [8]:
def get_possible_matches(matchmaking_matrix):
    player_count = matchmaking_matrix.shape[0]
    possible_matches = []
        
    for p1 in range(player_count):
        for p2 in range(player_count):
            if p1 < p2:
                if matchmaking_matrix[p1, p2] == 0:
                    possible_matches.append((p1, p2))
    
    return possible_matches

In [9]:
def get_player_match_count(matchmaking_matrix):
    return matchmaking_matrix.sum(axis=0)

In [10]:
def calculate_duplicate_matchmaking_rating(match_making_matrix, already_played_matrix):
    return int((already_played_matrix * match_making_matrix).sum() / 2)

In [11]:
def calculate_distance_rating(match_making_matrix, ranking_distance_matrix):
    return int((ranking_distance_matrix * match_making_matrix).sum() / 2)

In [12]:
def check_if_still_possible(player_match_count, possible_opponents, matches_per_round):
    possible_opponents_count = [len(possible_opponents) for p in possible_opponents]
    
    open_opponents_diff = np.array(possible_opponents_count) - (matches_per_round - np.array(player_match_count))
    
    not_enough_open_possibilities = np.min(open_opponents_diff) < 0
    to_much_matches = np.max(player_match_count) > matches_per_round
    
    return (not not_enough_open_possibilities) & (not to_much_matches)

In [13]:
input_path = r'matchmaking_input2.yml'

with open(input_path) as file:
    input_info = yaml.load(file, Loader=yaml.FullLoader)

In [14]:
player_dict, player_ids = create_player_dict(input_info['ranking'])

player_count = len(player_dict)
group_size = input_info['group_size']
matches_per_round = input_info['matches_per_round']
already_played = 0 if input_info["already_played"] is None else len(input_info["already_played"])

In [15]:
print(f'{player_count} Players\nRanking (account ids): {list(player_dict.keys())}\n--------')
print(f'Group size: {group_size}\nMatches per round: {matches_per_round}\nAlready played matches: {already_played}')

9 Players
Ranking (account ids): [6, 14, 1, 2, 10, 11, 8, 4, 5]
--------
Group size: 3
Matches per round: 4
Already played matches: 36


In [16]:
ranking_distance_matrix = create_ranking_distance_matrix(player_count)
already_played_matrix = create_already_played_matrix(player_count, input_info['already_played'])

In [17]:
print(f'Ranking distances matrix:\n{ranking_distance_matrix}\n')
print(f'Already played matrix:\n{already_played_matrix}')

Ranking distances matrix:
[[0 1 2 3 4 5 6 7 8]
 [1 0 1 2 3 4 5 6 7]
 [2 1 0 1 2 3 4 5 6]
 [3 2 1 0 1 2 3 4 5]
 [4 3 2 1 0 1 2 3 4]
 [5 4 3 2 1 0 1 2 3]
 [6 5 4 3 2 1 0 1 2]
 [7 6 5 4 3 2 1 0 1]
 [8 7 6 5 4 3 2 1 0]]

Already played matrix:
[[0 1 1 2 0 1 1 1 1]
 [1 0 1 1 1 1 1 1 1]
 [1 1 0 0 2 2 1 0 1]
 [2 1 0 0 1 0 1 2 1]
 [0 1 2 1 0 1 1 1 1]
 [1 1 2 0 1 0 1 1 1]
 [1 1 1 1 1 1 0 1 1]
 [1 1 0 2 1 1 1 0 1]
 [1 1 1 1 1 1 1 1 0]]


## Groups

In [18]:
groups = create_groups(player_count, group_size)

In [19]:
print(groups)

[[0, 1, 2], [3, 4, 5], [6, 7, 8]]


In [20]:
matchmaking_matrix, fixed_group_matches = initialize_matchmaking_matrix(player_count, groups)
possible_matches = get_possible_matches(matchmaking_matrix)
open_match_count = int((player_count * matches_per_round / 2) - fixed_group_matches)

In [21]:
print(f'Initial matchmaking matrix (group matches):\n{matchmaking_matrix}\n')
print(f'Possible open matches [{len(possible_matches)}]:\n{possible_matches}\n')
print(f'Fixed group matches: {fixed_group_matches}, Open match count: {open_match_count}')

Initial matchmaking matrix (group matches):
[[0 1 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 0]
 [0 0 0 1 0 1 0 0 0]
 [0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 1 1 0]]

Possible open matches [27]:
[(0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (3, 6), (3, 7), (3, 8), (4, 6), (4, 7), (4, 8), (5, 6), (5, 7), (5, 8)]

Fixed group matches: 9, Open match count: 9


In [22]:
initial_player_matches = get_player_match_count(matchmaking_matrix)

In [23]:
print(f'Initial player matches: {initial_player_matches}')

Initial player matches: [2 2 2 2 2 2 2 2 2]


In [24]:
initial_duplicate_matchmaking_rating = calculate_duplicate_matchmaking_rating(matchmaking_matrix, already_played_matrix)

In [25]:
print(f'Initial duplicate matchmaking rating: {initial_duplicate_matchmaking_rating}')

Initial duplicate matchmaking rating: 8


## Tree traversation

In [26]:
def is_partially_possible(matchmaking_matrix):
    new_player_match_count = get_player_match_count(matchmaking_matrix)
    new_possible_opponents = get_possible_opponents(matchmaking_matrix)
    
    return check_if_still_possible(new_player_match_count, new_possible_opponents, matches_per_round)

In [27]:
def get_random_matches(matchmaking_matrix, open_match_count):
    while True:
        mm_matrix = matchmaking_matrix.copy()
        
        for i in range(open_match_count):
            possible_matches = get_possible_matches(mm_matrix)
            
            if len(possible_matches) == 0:
                break
            
            p_id, o_id = random.choice(possible_matches)
            
            mm_matrix[p_id, o_id] = 1
            mm_matrix[o_id, p_id] = 1
            
            if not is_partially_possible(mm_matrix):
                break
            elif i == open_match_count-1:
                return mm_matrix

In [28]:
folder = r'F:\Magic\MoFLeague\scripts\matchmaking_9'

for i in range(100):
    mm = get_random_matches(matchmaking_matrix, open_match_count)
    mm_id = int(''.join(mm.ravel().astype(dtype='str')), 2)
    save_path = os.path.join(folder, f'{mm_id}.npy')
    
    if not os.path.isfile(save_path):
        np.save(save_path, mm)

KeyboardInterrupt: 

In [39]:
calculate_duplicate_matchmaking_rating(mm, already_played_matrix)

3

In [43]:
calculate_distance_rating(mm, ranking_distance_matrix)

76

In [47]:
binary_string = ''.join(mm.ravel().astype(dtype='str'))
print(binary_string)
print(int(binary_string, 2))

011000010010101000001010110110000000001011001000001101100000000110000101000010011100100000101001010100110000000001100011110000000101000001010110
8464198008663929798607344352599424981028950


In [35]:
mm

array([[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0],
       [1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1],
       [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1],
       [0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1],
       [0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0]])

In [37]:
mm

array([[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0],
       [1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1],
       [0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0]])

In [63]:
def get_random_combination(matchmaking_matrix, possible_opponents):
    mm_matrix = matchmaking_matrix.copy()
    
    while True:
        selected_matches = []
        
        for p_id in range(len(possible_opponents)):
            o_id = random.choice(possible_opponents[p_id])
            selected_matches.append(o_id)

            if not is_partially_possible(selected_matches, mm_matrix):
                break
            elif p_id == matchmaking_matrix.shape[0]-1:
                return selected_matches

In [65]:
for i in range(10):
    print(get_random_combination(matchmaking_matrix, possible_opponents))

[3, 5, 11, 1, 10, 8, 9, 0, 4, 7, 2, 6]
[5, 9, 8, 0, 11, 7, 10, 3, 1, 4, 2, 6]
[5, 3, 11, 2, 8, 9, 4, 1, 10, 6, 7, 0]


KeyboardInterrupt: 

In [24]:
pools = [tuple(pool) for pool in possible_opponents]

In [25]:
pools

[(3, 4, 5, 6, 7, 8, 9, 10, 11),
 (3, 4, 5, 6, 7, 8, 9, 10, 11),
 (3, 4, 5, 6, 7, 8, 9, 10, 11),
 (0, 1, 2, 6, 7, 8, 9, 10, 11),
 (0, 1, 2, 6, 7, 8, 9, 10, 11),
 (0, 1, 2, 6, 7, 8, 9, 10, 11),
 (0, 1, 2, 3, 4, 5, 9, 10, 11),
 (0, 1, 2, 3, 4, 5, 9, 10, 11),
 (0, 1, 2, 3, 4, 5, 9, 10, 11),
 (0, 1, 2, 3, 4, 5, 6, 7, 8),
 (0, 1, 2, 3, 4, 5, 6, 7, 8),
 (0, 1, 2, 3, 4, 5, 6, 7, 8)]

In [50]:
result = [[]]

count = 0

for pool in pools[:7]:
    result = [x+[y] for x in result for y in pool if is_partially_possible(x+[y], matchmaking_matrix)]    

KeyboardInterrupt: 

In [44]:
len(result)

1803312

In [48]:
result[5412:5418]

[[3, 4, 7, 7, 9, 8, 2],
 [3, 4, 7, 7, 9, 8, 5],
 [3, 4, 7, 7, 9, 8, 9],
 [3, 4, 7, 7, 9, 8, 10],
 [3, 4, 7, 7, 9, 8, 11],
 [3, 4, 7, 7, 9, 9, 0]]

In [29]:
possible_opponents

[[3, 4, 5, 6, 7, 8, 9, 10, 11],
 [3, 4, 5, 6, 7, 8, 9, 10, 11],
 [3, 4, 5, 6, 7, 8, 9, 10, 11],
 [0, 1, 2, 6, 7, 8, 9, 10, 11],
 [0, 1, 2, 6, 7, 8, 9, 10, 11],
 [0, 1, 2, 6, 7, 8, 9, 10, 11],
 [0, 1, 2, 3, 4, 5, 9, 10, 11],
 [0, 1, 2, 3, 4, 5, 9, 10, 11],
 [0, 1, 2, 3, 4, 5, 9, 10, 11],
 [0, 1, 2, 3, 4, 5, 6, 7, 8],
 [0, 1, 2, 3, 4, 5, 6, 7, 8],
 [0, 1, 2, 3, 4, 5, 6, 7, 8]]

In [55]:
possible_combinations = list(itertools.product(*possible_opponents[:2]))
print(len(possible_combinations))
print(possible_combinations[0])

81
(3, 3)


In [ ]:
P_ID = 0
O_ID = 0

matchmaking_matrix[P_ID, possible_opponents[P_ID][O_ID]] = 1
matchmaking_matrix[possible_opponents[P_ID][O_ID], P_ID] = 1

new_player_match_count = get_player_match_count(matchmaking_matrix)
new_possible_opponents = get_possible_opponents(matchmaking_matrix)

still_possible = check_if_still_possible(new_player_match_count, new_possible_opponents, matches_per_round)

In [ ]:
print(new_player_match_count)
print(new_possible_opponents)
print(still_possible)